<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# IA Framework.
## Lab 1  - Introduction to Pyspark.
#### Part 2 Basic Statistics and Logistic Regression with <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> and  [MLlib](https://spark.apache.org/mllib/)

**Resume**: This notebook continues the introduction to [Spark](https://spark.apache.org/) trough  [`PySpark`](http://spark.apache.org/docs/latest/api/python/) API. We will see how to sample RDD, an introduction to the MlLib library, descriptive statistics and basic uni and multi dimensional(s) statistics.

## Context and Dataset

This notebook will continue to used  [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) dataset

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [2]:
DATA_PATH="" 
data_file = DATA_PATH+"kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

## RDD's sampling
They are two  `functions` available for sampling a RDD in spark.* `sample` and `takeSample`. 

In [13]:
raw_data_sample = raw_data.takeSample(False, 4000, seed=1234)
len(raw_data_sample)

4000

In [15]:
raw_data_sample = raw_data.sample(False, 0.1, seed=1234)
raw_data_sample.count()

49493

**Question** What is the difference between these two functions? Are the sample identical?

**Réponse**
- takeSample (list python) : construit un dataset d'une longueur fixé en prenant aléatoirement des valeurs du rdd
- sample (RDD) : construit un dataset d'une longueur égale au pourcentage * longueur du RDD passée en paramètre

The use of the `sample` function allows to quickly estimate some values on a subsample of a huge dataset.

In the cells below, we estimate the rate of normal interaction on the sample and on the all dataset.

In [16]:
import time
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_size = raw_data_sample.count()
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)
t0 = time.time()
sample_normal_tags_count = sample_normal_tags.count()
tt = time.time() - t0
sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print("Normal interaction rate is {}".format(round(sample_normal_ratio,3)))
print("Running time: {} secondes".format(round(tt,3)))

Normal interaction rate is 0.195
Running time: 0.758 secondes


In [17]:
raw_data_items = raw_data.map(lambda x: x.split(","))
total_size = raw_data.count()
normal_tags = raw_data_items.filter(lambda x: "normal." in x)
t0 = time.time()
normal_tags_count = normal_tags.count()
tt = time.time() - t0
normal_ratio = normal_tags_count / float(total_size)
print("Normal interaction rate is {}".format(round(sample_normal_ratio,3)))
print("Running time: {} secondes".format(round(tt,3)))

Normal interaction rate is 0.195
Running time: 1.637 secondes


###  Data munging
As described in the python [tutoriel](https://github.com/wikistat/Intro-Python) dedicated to data munging with `pandas`, data preprocessing is an essential part before analysis and modelling the data. Extraction, filtering, sampling, data completion, correction, anomaly detection, normalisation, features selection, matching, etc.

Most of these steps are unidimensional and can easily be distributed.


### [MLlib](http://spark.apache.org/docs/latest/ml-guide.html)


*MLlib* is a RDD-based library.  
Another library, *SparkML* is developed by Spark and is based on DataFrame (see part 3).

*SparkML* will soon completely replace *MlLib* library, but so far, some functionalities (especially for basic statistics) are available only on MLlib.
This two libraries will coexist until spark 3 is released.

The only up-to-date documentation is the official online [documentation](https://spark.apache.org/docs/latest/mllib-guide.html).




We first build a RDD that contains only numerical values.

In [18]:
import numpy as np
def parse_interaction(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data = raw_data.map(parse_interaction)
vector_data.take(2)

[array([0.00e+00, 1.81e+02, 5.45e+03, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 9.00e+00, 9.00e+00,
        1.00e+00, 0.00e+00, 1.10e-01, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00]),
 array([0.00e+00, 2.39e+02, 4.86e+02, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 8.00e+00, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 1.90e+01, 1.90e+01,
        1.00e+00, 0.00e+00, 5.00e-02, 0.00e+00, 0.00e+00, 0.00e+00,
        0.00e+00, 0.00e+00])]

## Basic Statistics
### Summary statistics

[`colStats`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.Statistics.colStats)' MLlib function enable to compute unidimensionals statistics for each columns of the `RDD[Vector]`. It returns a[`MultivariateStatisticalSummary`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.MultivariateStatisticalSummary) object.

In [19]:
from pyspark.mllib.stat import Statistics
from math import sqrt 

# Compute column summary statistics.
summary = Statistics.colStats(vector_data)

print("Statistique des durées")
print(" Moyenne: {}".format(round(summary.mean()[0],3)))
print(" Ecart type: {}".format(round(sqrt(summary.variance()[0]),3)))
print(" Valeur max: {}".format(round(summary.max()[0],3)))
print(" Valeur min: {}".format(round(summary.min()[0],3)))
print(" Nombre de valeurs: {}".format(summary.count()))
print(" Nombre de valeurs non nulles: {}".format(summary.numNonzeros()[0]))

Statistique des durées
 Moyenne: 47.979
 Ecart type: 707.746
 Valeur max: 58329.0
 Valeur min: 0.0
 Nombre de valeurs: 494021
 Nombre de valeurs non nulles: 12350.0


### By label

In [20]:
def parse_interaction_with_key(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return (line_split[41], np.array([float(x) for x in clean_line_split]))

def summary_by_label(raw_data, label):
    label_vector_data = raw_data.map(parse_interaction_with_key).filter(lambda x: x[0]==label)
    return Statistics.colStats(label_vector_data.values())


In [21]:
normal_sum = summary_by_label(raw_data, "normal.")

print("Duration Statistics for label: {}".format("normal"))
print(" Mean: {}".format(normal_sum.mean()[0],3))
print(" St. deviation: {}".format(round(sqrt(normal_sum.variance()[0]),3)))
print(" Max value: {}".format(round(normal_sum.max()[0],3)))
print(" Min value: {}".format(round(normal_sum.min()[0],3)))
print(" Total value count: {}".format(normal_sum.count()))
print(" Number of non-zero values: {}".format(normal_sum.numNonzeros()[0]))

Duration Statistics for label: normal
 Mean: 216.65732231336938
 St. deviation: 1359.213
 Max value: 58329.0
 Min value: 0.0
 Total value count: 97278
 Number of non-zero values: 11690.0


### For all label

In [22]:
label_list = ["back.","buffer_overflow.","ftp_write.","guess_passwd.",
              "imap.","ipsweep.","land.","loadmodule.","multihop.",
              "neptune.","nmap.","normal.","perl.","phf.","pod.","portsweep.",
              "rootkit.","satan.","smurf.","spy.","teardrop.","warezclient.",
              "warezmaster."]

In [23]:
stats_by_label =[(label, summary_by_label(raw_data, label)) for label in label_list]

In [24]:
import pandas as pd
#Display results with `pandas`.
def get_variable_stats_df(stats_by_label, column_i):
    column_stats_by_label = [
        (stat[0], np.array([float(stat[1].mean()[column_i]), float(sqrt(stat[1].variance()[column_i])), float(stat[1].min()[column_i]), float(stat[1].max()[column_i]), int(stat[1].count())])) 
        for stat in stats_by_label
    ]
    return pd.DataFrame.from_items(column_stats_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')

In [25]:
print("Duration statistics, by label")
get_variable_stats_df(stats_by_label,0)

Duration statistics, by label


/home/vavalm/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  


Mean      Std Dev    Min      Max     Count
back.                0.128915     1.110062    0.0     14.0    2203.0
buffer_overflow.    91.700000    97.514685    0.0    321.0      30.0
ftp_write.          32.375000    47.449033    0.0    134.0       8.0
guess_passwd.        2.716981    11.879811    0.0     60.0      53.0
imap.                6.000000    14.174240    0.0     41.0      12.0
ipsweep.             0.034483     0.438439    0.0      7.0    1247.0
land.                0.000000     0.000000    0.0      0.0      21.0
loadmodule.         36.222222    41.408869    0.0    103.0       9.0
multihop.          184.000000   253.851006    0.0    718.0       7.0
neptune.             0.000000     0.000000    0.0      0.0  107201.0
nmap.                0.000000     0.000000    0.0      0.0     231.0
normal.            216.657322  1359.213469    0.0  58329.0   97278.0
perl.               41.333333    14.843629   25.0     54.0       3.0
phf.                 4.500000     5.744563    0.0     12.0       4.0
pod.                 0.000000     0.000000    0.0      0.0     264.0
portsweep.        1915.299038  7285.125159    0.0  42448.0    1040.0
rootkit.           100.800000   216.185003    0.0    708.0      10.0
satan.               0.040277     0.522433    0.0     11.0    1589.0
smurf.               0.000000     0.000000    0.0      0.0  280790.0
spy.               318.000000    26.870058  299.0    337.0       2.0
teardrop.            0.000000     0.000000    0.0      0.0     979.0
warezclient.       615.257843  2207.694966    0.0  15168.0    1020.0
warezmaster.        15.050000    33.385271    0.0    156.0      20.0

In [26]:
print("src_bytes statistics, by label")
get_variable_stats_df(stats_by_label,1)

src_bytes statistics, by label


/home/vavalm/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  


Mean       Std Dev      Min          Max     Count
back.              54156.355878  3.159360e+03  13140.0      54540.0    2203.0
buffer_overflow.    1400.433333  1.337133e+03      0.0       6274.0      30.0
ftp_write.           220.750000  2.677476e+02      0.0        676.0       8.0
guess_passwd.        125.339623  3.037860e+00    104.0        126.0      53.0
imap.                347.583333  6.299260e+02      0.0       1492.0      12.0
ipsweep.              10.083400  5.231658e+00      0.0         18.0    1247.0
land.                  0.000000  0.000000e+00      0.0          0.0      21.0
loadmodule.          151.888889  1.277453e+02      0.0        302.0       9.0
multihop.            435.142857  5.409604e+02      0.0       1412.0       7.0
neptune.               0.000000  0.000000e+00      0.0          0.0  107201.0
nmap.                 24.116883  5.941987e+01      0.0        207.0     231.0
normal.             1157.047524  3.422612e+04      0.0    2194619.0   97278.0
perl.                265.666667  4.932883e+00    260.0        269.0       3.0
phf.                  51.000000  0.000000e+00     51.0         51.0       4.0
pod.                1462.651515  1.250980e+02    564.0       1480.0     264.0
portsweep.        666707.436538  2.150067e+07      0.0  693375640.0    1040.0
rootkit.             294.700000  5.385782e+02      0.0       1727.0      10.0
satan.                 1.337319  4.294620e+01      0.0       1710.0    1589.0
smurf.               935.772300  2.000224e+02    520.0       1032.0  280790.0
spy.                 174.500000  8.838835e+01    112.0        237.0       2.0
teardrop.             28.000000  0.000000e+00     28.0         28.0     979.0
warezclient.      300219.562745  1.200905e+06     30.0    5135678.0    1020.0
warezmaster.          49.300000  2.121551e+02      0.0        950.0      20.0

### Correlation
`corr` applies Spearman (rangs) and Pearson correlation/

In [27]:
from pyspark.mllib.stat import Statistics 
correlation_matrix = Statistics.corr(vector_data, method="pearson")

In [28]:
import pandas as pd
pd.set_option('display.max_columns', 50)
col_names = ["duration","src_bytes","dst_bytes","land","wrong_fragment",
             "urgent","hot","num_failed_logins","logged_in","num_compromised",
             "root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds",
             "is_hot_login","is_guest_login","count","srv_count","serror_rate",
             "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

corr_df = pd.DataFrame(correlation_matrix, index=col_names, columns=col_names)
corr_df

duration  src_bytes  dst_bytes      land  \
duration                     1.000000   0.004258   0.005440 -0.000452   
src_bytes                    0.004258   1.000000  -0.000002 -0.000020   
dst_bytes                    0.005440  -0.000002   1.000000 -0.000175   
land                        -0.000452  -0.000020  -0.000175  1.000000   
wrong_fragment              -0.003235  -0.000139  -0.001254 -0.000318   
urgent                       0.003786  -0.000005   0.016288 -0.000017   
hot                          0.013213   0.004483   0.004365 -0.000295   
num_failed_logins            0.005239  -0.000027   0.049330 -0.000065   
logged_in                   -0.017265   0.001701   0.047814 -0.002784   
num_compromised              0.058095   0.000119   0.023298 -0.000038   
root_shell                   0.021340  -0.000022   0.031680 -0.000070   
su_attempted                 0.055853  -0.000010   0.075656 -0.000031   
num_root                     0.056766  -0.000010   0.020746 -0.000038   
num_file_creations           0.074562   0.000013   0.004958 -0.000075   
num_shells                  -0.000169   0.000005   0.000144 -0.000066   
num_access_files             0.025661  -0.000052   0.008746 -0.000184   
num_outbound_cmds                 NaN        NaN        NaN       NaN   
is_hot_login                      NaN        NaN        NaN       NaN   
is_guest_login               0.023424  -0.000082   0.001289 -0.000249   
count                       -0.105153  -0.003098  -0.040373 -0.010260   
srv_count                   -0.080250  -0.002501  -0.030544 -0.007886   
serror_rate                 -0.031416   0.001558  -0.011908  0.013898   
srv_serror_rate             -0.031378   0.001114  -0.011930  0.014422   
rerror_rate                  0.012053   0.000591  -0.006166 -0.000777   
srv_rerror_rate              0.012106   0.001379  -0.005808 -0.001659   
same_srv_rate                0.021771  -0.001860   0.014002  0.002286   
diff_srv_rate                0.051800   0.006207  -0.005702  0.002282   
srv_diff_host_rate          -0.011790  -0.000015   0.008135  0.036985   
dst_host_count               0.010074  -0.001743  -0.048869 -0.023671   
dst_host_srv_count          -0.117515  -0.003212  -0.005850 -0.011587   
dst_host_same_srv_rate      -0.118458  -0.002052   0.007058  0.001984   
dst_host_diff_srv_rate       0.406233   0.000578  -0.005314 -0.000333   
dst_host_same_src_port_rate  0.042642  -0.000724  -0.020143  0.003799   
dst_host_srv_diff_host_rate -0.006983   0.001186   0.008707  0.083320   
dst_host_serror_rate        -0.030400  -0.000718  -0.011334  0.012658   
dst_host_srv_serror_rate    -0.030612   0.001122  -0.011235  0.007795   
dst_host_rerror_rate         0.006739  -0.000393  -0.005000 -0.001511   
dst_host_srv_rerror_rate     0.010465   0.001328  -0.005471 -0.001665   

                             wrong_fragment    urgent       hot  \
duration                          -0.003235  0.003786  0.013213   
src_bytes                         -0.000139 -0.000005  0.004483   
dst_bytes                         -0.001254  0.016288  0.004365   
land                              -0.000318 -0.000017 -0.000295   
wrong_fragment                     1.000000 -0.000123 -0.002106   
urgent                            -0.000123  1.000000  0.000356   
hot                               -0.002106  0.000356  1.000000   
num_failed_logins                 -0.000467  0.141996  0.008740   
logged_in                         -0.019908  0.006164  0.105305   
num_compromised                   -0.000271  0.014285  0.007348   
root_shell                        -0.000504  0.034790  0.024065   
su_attempted                      -0.000223 -0.000012 -0.000206   
num_root                          -0.000269  0.009476  0.000998   
num_file_creations                -0.000536  0.015211  0.025247   
num_shells                        -0.000473 -0.000026  0.006373   
num_access_files                  -0.001319  0.020068  0.001902   
num_outbound_cmds                       Na

Most correlated features

In [30]:
# Une variable bouléenne est True en cas de forte corrélation
highly_correlated_df = (abs(corr_df) > .8) & (abs(corr_df) < 1.0)
# Extraction des noms des variables
correlated_vars_index = (highly_correlated_df==True).any()
correlated_var_names = correlated_vars_index[correlated_vars_index==True].index
highly_correlated_df.loc[correlated_var_names,correlated_var_names]

hot  num_compromised  num_root  is_guest_login  \
hot                          False            False     False            True   
num_compromised              False            False      True           False   
num_root                     False             True     False           False   
is_guest_login                True            False     False           False   
count                        False            False     False           False   
srv_count                    False            False     False           False   
serror_rate                  False            False     False           False   
srv_serror_rate              False            False     False           False   
rerror_rate                  False            False     False           False   
srv_rerror_rate              False            False     False           False   
same_srv_rate                False            False     False           False   
dst_host_srv_count           False            False     False           False   
dst_host_same_srv_rate       False            False     False           False   
dst_host_same_src_port_rate  False            False     False           False   
dst_host_serror_rate         False            False     False           False   
dst_host_srv_serror_rate     False            False     False           False   
dst_host_rerror_rate         False            False     False           False   
dst_host_srv_rerror_rate     False            False     False           False   

                             count  srv_count  serror_rate  srv_serror_rate  \
hot                          False      False        False            False   
num_compromised              False      False        False            False   
num_root                     False      False        False            False   
is_guest_login               False      False        False            False   
count                        False       True        False            False   
srv_count                     True      False        False            False   
serror_rate                  False      False        False             True   
srv_serror_rate              False      False         True            False   
rerror_rate                  False      False        False            False   
srv_rerror_rate              False      False        False            False   
same_srv_rate                False      False         True             True   
dst_host_srv_count           False      False        False            False   
dst_host_same_srv_rate       False      False        False            False   
dst_host_same_src_port_rate   True       True        False            False   
dst_host_serror_rate         False      False         True             True   
dst_host_srv_serror_rate     False      False         True             True   
dst_host_rerror_rate         False      False        False            False   
dst_host_srv_rerror_rate     False      False        False            False   

                             rerror_rate  srv_rerror_rate  same_srv_rate  \
hot                                False            False          False   
num_compromised                    False            False          False   
num_root                           False            False          False   
is_guest_login                     False            False          False   
count                              False            False          False   
srv_count                          False            False          False   
serror_rate                        False            False           True   
srv_serror_rate                    False            False           True   
rerror_rate                        False             True          False   
srv_rerror_rate                     True            False          False   
same_srv_rate                      False            False          False   
dst_host_srv_count                 False            False           True   
d

## Logistic Regression

**Warnings**: [J. A. Dianes](https://github.com/jadianes/spark-py-notebooks/blob/master/nb8-mllib-logit/nb8-mllib-logit.ipynb) uses the previous function to select the features according to their correlation before using them within logisticRegression.
This procedure is not recommended in Machine learning. It's better to used step-by-step method (*forward, backward, both*) and select the variable according to AIC or BIC criteria or using Lasso penalisation. This is what is implemented in MLlib library.

### Read the data.

We now use the complete [KDD cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) dataset. If you have memory issue, restart the kernel and start from here. You can also sample the data to make it run on a smallest dataset.

train dataset

In [31]:
import urllib.request
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", DATA_PATH+"data.gz")
train_data_file = DATA_PATH+"data.gz"
train_raw_data = sc.textFile(train_data_file)

print("Train data size is {}".format(train_raw_data.count()))

Train data size is 4898431


test dataset

In [32]:
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz", DATA_PATH+"corrected.gz")
test_data_file = DATA_PATH+"corrected.gz"
test_raw_data = sc.textFile(test_data_file)

print("Test data size is {}".format(test_raw_data.count()))

Test data size is 311029


### Labeled Point

In order to perform learning, you have to convert the data in the **Labeled Points** format.
This object takes the label and the features as inputs.
We define a function to convert the raw line to a LabeledPoint object.

In [33]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array

def parse_interaction(line):
    line_split = line.split(",")
    clean_line_split = line_split[0:1]+line_split[4:41]
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

In [36]:
test_data = test_raw_data.map(parse_interaction)

In [41]:
train_data = train_raw_data.map(parse_interaction)

###  Learning the model
Training the model is basically the same than with scikit learn.
Mllib proposes two optimisation [algorithms](https://spark.apache.org/docs/latest/mllib-linear-methods.html#logistic-regression):  *mini-batch gradient descent* and L-BFGS (L-BFGS which is supposed to run faster).

As in  *Scikit-learn*, *l2* (ridge) and *l1* (lasso) are available.

See full doc [here](https://spark.apache.org/docs/2.0.0/api/python/pyspark.mllib.html)

In [42]:
from time import time
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
t0 = time()
logit_model = LogisticRegressionWithLBFGS.train(train_data)
tt = time() - t0
print("Apprentissage en {} seconds".format(round(tt,3)))

Apprentissage en 1823.37 seconds


### Error Estimation
The `map` function enables to predict the attack for each entry

In [43]:
labels_and_preds = test_data.map(lambda p: (p.label, logit_model.predict(p.features)))

We can then compute the error

In [44]:
t0 = time()
test_accuracy = labels_and_preds.filter(lambda x: x[0] == x[1]).count() / float(test_data.count())
erreur=1-test_accuracy
tt = time() - t0
print("Calcul en {} secondes. Le taux d'erreur est {}".format(round(tt,3), round(erreur,4)))

Calcul en 11.688 secondes. Le taux d'erreur est 0.1374


**Exercices**

- Qualitative variable such as `protocol` or `service` has been removed for simplicity. Add them as indicatrices and re-train the model.(*dummy variables*)
- Try to [optimize](https://spark.apache.org/docs/latest/ml-tuning.html) the model with cross validation by trying various parameters.
- Use other algorithm such as  [RandomForest](https://spark.apache.org/docs/latest/mllib-ensembles.html#random-forests) on this problem.